# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:**  My Architecture start with EncoderCNN where resnet50 pretrained model was used to obtained feature which was of shape(10,2048,1,1) which was then flatten to shape(10,2048) and to standarize the shape for input to DecoderRNN it was send to Linear model with output in shape (10, embed_size).

Then, DecoderRNN was expected to get inputs from both features from EncoderCNN and captions which could be obtained by following way.
1.All captions were taken except for <end> tokens as DecoderRNN takes previous outputs as end token.
2.Then it was passed through Embedding Layer to make them in shape of embed_size as features from EncoderCNN was
in embed_size. So after passing Embedding Layer, caption will be of shape (batch_size,caption_size-1,embed_size)
3.Now we added additional dimension to the features as it is 2D and caption is 3D so concat is not possible so we added dimension in dim=1 as other two have dimension i.e. batch_size and embed_size.
4.After concat input shape become (batch_size,caption_size,embed_size), it is send to lstm with batch_first=True
and gives output of shape (batch_size,caption_size,hidden_size)
5.Then as we want our output to be in reference to vocabulary we pass that output to linear layer to convert that output to (batch_size,caption_size,vocab_size
    
With above architecture in mind: 
Initially i choose following hyperparameters randomly based on earlier lesson tuning hyperparameter in LSTM.
batch_size = 10          #as all previous visualization was done with batch_size=10     
vocab_threshold = 4      (#as previously we have seen with value 4 and 5, i assumed vocab with words occuring 4 will be good for model as important words will not be excluded nor less important words will be included)
vocab_from_file = True    (#as in previous notebook atlast vocab was found with threshold=4 so for fast implementation it was choose True)
embed_size = 256           (#from lesson tuning hypermeters i learned embed_size to be good around 50 to 200 so choose randomly to check output)
hidden_size = 512          (#hidden size also chose randomly based on previously learning)
num_epochs = 3             (#as data were large epoch 3 would be okay as instructed)
save_every = 1             
print_every = 100          
log_file = 'training_log.txt' 

Then, from show attend and tell paper i found out model will perform better on bigger batch_size. So, i increased it to batch_size = 64

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I left it as it is. As all edges were resized to value  256 which would be sufficient for CNN with input width and height 224 which is standard for Resnet50 CNN. And for Normalization these default normalized value were considered after training on huge datasets by Resnet50 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I selected params as mentioned above as i found that to be relevant. Because in decoder each layers weight will be trained to reach their best value for best prediction.But in encoder as we have used pretrained CNN with its best weight on certain field i felt it should be left as it and only embeded layers weight can be trained.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**  Here i chose Adam as it gets better with epochs and doesnot much depends on learning rate(lr). However i first tried with lr=0.01 with intial hyperparameters but later change it to lr =0.001

In [ ]:
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64         # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters())+list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params,lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.04s)
creating index...


  0%|          | 789/414113 [00:00<01:49, 3781.39it/s]

index created!
Obtaining caption lengths...


 46%|████▋     | 192323/414113 [00:43<00:49, 4470.52it/s]

In [4]:
from workspace_utils import active_session

NameError: name 'encoder_file' is not defined

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})
with active_session():
    
    for epoch in range(1, num_epochs+1):
    
        for i_step in range(1, total_step+1):
        
            if time.time() - old_time > 60:
                old_time = time.time()
                requests.request("POST", 
                                 "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                                 headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
            indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
            new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
            data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
            images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            captions = captions.to(device)
        
        # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
            features = encoder(images)
            outputs = decoder(features, captions)
        
        # Calculate the batch loss.
            loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
            loss.backward()
        
        # Update the parameters in the optimizer.
            optimizer.step()
            
        # Get training statistics.
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
            print('\r' + stats, end="")
            sys.stdout.flush()
        
        # Print training statistics to file.
            f.write(stats + '\n')
            f.flush()
        
        # Print training statistics (on different line).
            if i_step % print_every == 0:
                print('\r' + stats)
            
    # Save the weights.
        if epoch % save_every == 0:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
    f.close()

sel length 15
Epoch [1/3], Step [1/6471], Loss: 9.2116, Perplexity: 10012.4163sel length 12
Epoch [1/3], Step [2/6471], Loss: 9.0633, Perplexity: 8632.5924sel length 11
Epoch [1/3], Step [3/6471], Loss: 8.8782, Perplexity: 7173.7947sel length 10
Epoch [1/3], Step [4/6471], Loss: 8.6037, Perplexity: 5452.0106sel length 12
Epoch [1/3], Step [5/6471], Loss: 8.2817, Perplexity: 3950.8342sel length 11
Epoch [1/3], Step [6/6471], Loss: 7.6633, Perplexity: 2128.7150sel length 9
Epoch [1/3], Step [7/6471], Loss: 6.9253, Perplexity: 1017.6798sel length 9
Epoch [1/3], Step [8/6471], Loss: 6.3168, Perplexity: 553.7717sel length 10
Epoch [1/3], Step [9/6471], Loss: 5.5521, Perplexity: 257.7795sel length 15
Epoch [1/3], Step [10/6471], Loss: 5.4496, Perplexity: 232.6644sel length 10
Epoch [1/3], Step [11/6471], Loss: 5.1820, Perplexity: 178.0393sel length 7
Epoch [1/3], Step [12/6471], Loss: 5.4622, Perplexity: 235.6090sel length 9
Epoch [1/3], Step [13/6471], Loss: 4.7658, Perplexity: 117.4277sel 

Epoch [1/3], Step [214/6471], Loss: 3.3989, Perplexity: 29.9314sel length 13
Epoch [1/3], Step [215/6471], Loss: 3.2255, Perplexity: 25.1657sel length 12
Epoch [1/3], Step [216/6471], Loss: 3.4321, Perplexity: 30.9401sel length 11
Epoch [1/3], Step [217/6471], Loss: 3.5225, Perplexity: 33.8701sel length 9
Epoch [1/3], Step [218/6471], Loss: 3.7887, Perplexity: 44.2010sel length 9
Epoch [1/3], Step [219/6471], Loss: 3.5602, Perplexity: 35.1709sel length 13
Epoch [1/3], Step [220/6471], Loss: 3.5240, Perplexity: 33.9199sel length 9
Epoch [1/3], Step [221/6471], Loss: 3.7011, Perplexity: 40.4926sel length 10
Epoch [1/3], Step [222/6471], Loss: 3.5500, Perplexity: 34.8142sel length 10
Epoch [1/3], Step [223/6471], Loss: 3.3713, Perplexity: 29.1163sel length 13
Epoch [1/3], Step [224/6471], Loss: 3.5785, Perplexity: 35.8200sel length 11
Epoch [1/3], Step [225/6471], Loss: 3.6091, Perplexity: 36.9328sel length 10
Epoch [1/3], Step [226/6471], Loss: 3.2791, Perplexity: 26.5513sel length 10
Ep

Epoch [1/3], Step [426/6471], Loss: 3.2316, Perplexity: 25.3197sel length 9
Epoch [1/3], Step [427/6471], Loss: 3.2235, Perplexity: 25.1146sel length 11
Epoch [1/3], Step [428/6471], Loss: 3.2808, Perplexity: 26.5979sel length 14
Epoch [1/3], Step [429/6471], Loss: 3.2238, Perplexity: 25.1225sel length 17
Epoch [1/3], Step [430/6471], Loss: 3.7138, Perplexity: 41.0101sel length 9
Epoch [1/3], Step [431/6471], Loss: 3.5089, Perplexity: 33.4119sel length 29
Epoch [1/3], Step [432/6471], Loss: 4.6339, Perplexity: 102.9187sel length 13
Epoch [1/3], Step [433/6471], Loss: 3.1347, Perplexity: 22.9821sel length 13
Epoch [1/3], Step [434/6471], Loss: 3.3338, Perplexity: 28.0453sel length 11
Epoch [1/3], Step [435/6471], Loss: 2.9236, Perplexity: 18.6076sel length 14
Epoch [1/3], Step [436/6471], Loss: 3.3627, Perplexity: 28.8658sel length 7
Epoch [1/3], Step [437/6471], Loss: 3.9621, Perplexity: 52.5680sel length 10
Epoch [1/3], Step [438/6471], Loss: 3.1511, Perplexity: 23.3624sel length 14
E

Epoch [1/3], Step [638/6471], Loss: 3.2489, Perplexity: 25.7616sel length 20
Epoch [1/3], Step [639/6471], Loss: 3.8444, Perplexity: 46.7326sel length 12
Epoch [1/3], Step [640/6471], Loss: 2.9832, Perplexity: 19.7513sel length 10
Epoch [1/3], Step [641/6471], Loss: 3.2857, Perplexity: 26.7287sel length 9
Epoch [1/3], Step [642/6471], Loss: 3.0731, Perplexity: 21.6099sel length 11
Epoch [1/3], Step [643/6471], Loss: 3.1122, Perplexity: 22.4711sel length 11
Epoch [1/3], Step [644/6471], Loss: 2.9991, Perplexity: 20.0683sel length 19
Epoch [1/3], Step [645/6471], Loss: 3.5770, Perplexity: 35.7674sel length 13
Epoch [1/3], Step [646/6471], Loss: 3.0817, Perplexity: 21.7945sel length 9
Epoch [1/3], Step [647/6471], Loss: 3.1739, Perplexity: 23.9002sel length 20
Epoch [1/3], Step [648/6471], Loss: 4.0372, Perplexity: 56.6654sel length 8
Epoch [1/3], Step [649/6471], Loss: 3.2399, Perplexity: 25.5322sel length 10
Epoch [1/3], Step [650/6471], Loss: 3.2096, Perplexity: 24.7699sel length 12
Ep

Epoch [1/3], Step [850/6471], Loss: 2.7798, Perplexity: 16.1154sel length 18
Epoch [1/3], Step [851/6471], Loss: 3.4493, Perplexity: 31.4783sel length 16
Epoch [1/3], Step [852/6471], Loss: 3.1977, Perplexity: 24.4771sel length 9
Epoch [1/3], Step [853/6471], Loss: 2.9479, Perplexity: 19.0663sel length 12
Epoch [1/3], Step [854/6471], Loss: 2.8396, Perplexity: 17.1097sel length 12
Epoch [1/3], Step [855/6471], Loss: 2.8036, Perplexity: 16.5036sel length 11
Epoch [1/3], Step [856/6471], Loss: 2.9004, Perplexity: 18.1818sel length 11
Epoch [1/3], Step [857/6471], Loss: 2.8445, Perplexity: 17.1936sel length 9
Epoch [1/3], Step [858/6471], Loss: 3.0561, Perplexity: 21.2441sel length 11
Epoch [1/3], Step [859/6471], Loss: 2.7241, Perplexity: 15.2419sel length 14
Epoch [1/3], Step [860/6471], Loss: 2.8507, Perplexity: 17.2994sel length 17
Epoch [1/3], Step [861/6471], Loss: 3.3303, Perplexity: 27.9460sel length 12
Epoch [1/3], Step [862/6471], Loss: 2.9543, Perplexity: 19.1889sel length 13
E

...

Epoch [3/3], Step [4274/6471], Loss: 2.4399, Perplexity: 11.4714sel length 11
Epoch [3/3], Step [4275/6471], Loss: 1.8091, Perplexity: 6.1050sel length 9
Epoch [3/3], Step [4276/6471], Loss: 2.0046, Perplexity: 7.4232sel length 9
Epoch [3/3], Step [4277/6471], Loss: 1.9607, Perplexity: 7.1045sel length 8
Epoch [3/3], Step [4278/6471], Loss: 2.1949, Perplexity: 8.9792sel length 10
Epoch [3/3], Step [4279/6471], Loss: 1.7625, Perplexity: 5.8268sel length 9
Epoch [3/3], Step [4280/6471], Loss: 2.0009, Perplexity: 7.3958sel length 9
Epoch [3/3], Step [4281/6471], Loss: 1.8957, Perplexity: 6.6571sel length 10
Epoch [3/3], Step [4282/6471], Loss: 1.8468, Perplexity: 6.3393sel length 10
Epoch [3/3], Step [4283/6471], Loss: 1.8248, Perplexity: 6.2014sel length 11
Epoch [3/3], Step [4284/6471], Loss: 1.8211, Perplexity: 6.1787sel length 9
Epoch [3/3], Step [4285/6471], Loss: 1.8755, Perplexity: 6.5242sel length 11
Epoch [3/3], Step [4286/6471], Loss: 1.9692, Perplexity: 7.1651sel length 9
Epoch

Epoch [3/3], Step [4486/6471], Loss: 1.8571, Perplexity: 6.4051sel length 10
Epoch [3/3], Step [4487/6471], Loss: 1.6931, Perplexity: 5.4365sel length 9
Epoch [3/3], Step [4488/6471], Loss: 2.0089, Perplexity: 7.4555sel length 10
Epoch [3/3], Step [4489/6471], Loss: 2.0840, Perplexity: 8.0365sel length 11
Epoch [3/3], Step [4490/6471], Loss: 1.9647, Perplexity: 7.1326sel length 9
Epoch [3/3], Step [4491/6471], Loss: 1.9660, Perplexity: 7.1422sel length 10
Epoch [3/3], Step [4492/6471], Loss: 1.8222, Perplexity: 6.1857sel length 11
Epoch [3/3], Step [4493/6471], Loss: 1.9196, Perplexity: 6.8183sel length 9
Epoch [3/3], Step [4494/6471], Loss: 2.0991, Perplexity: 8.1587sel length 11
Epoch [3/3], Step [4495/6471], Loss: 1.8276, Perplexity: 6.2188sel length 11
Epoch [3/3], Step [4496/6471], Loss: 1.8070, Perplexity: 6.0922sel length 12
Epoch [3/3], Step [4497/6471], Loss: 1.9450, Perplexity: 6.9934sel length 9
Epoch [3/3], Step [4498/6471], Loss: 2.0642, Perplexity: 7.8794sel length 8
Epoc

Epoch [3/3], Step [4698/6471], Loss: 1.7785, Perplexity: 5.9207sel length 8
Epoch [3/3], Step [4699/6471], Loss: 2.1419, Perplexity: 8.5153sel length 12
Epoch [3/3], Step [4700/6471], Loss: 1.8130, Perplexity: 6.1291
sel length 12
Epoch [3/3], Step [4701/6471], Loss: 1.9020, Perplexity: 6.6991sel length 9
Epoch [3/3], Step [4702/6471], Loss: 2.0096, Perplexity: 7.4604sel length 13
Epoch [3/3], Step [4703/6471], Loss: 2.0149, Perplexity: 7.4998sel length 12
Epoch [3/3], Step [4704/6471], Loss: 2.1092, Perplexity: 8.2419sel length 10
Epoch [3/3], Step [4705/6471], Loss: 1.7593, Perplexity: 5.8083sel length 10
Epoch [3/3], Step [4706/6471], Loss: 2.0706, Perplexity: 7.9297sel length 15
Epoch [3/3], Step [4707/6471], Loss: 2.2348, Perplexity: 9.3449sel length 15
Epoch [3/3], Step [4708/6471], Loss: 2.2906, Perplexity: 9.8806sel length 11
Epoch [3/3], Step [4709/6471], Loss: 1.8613, Perplexity: 6.4320sel length 13
Epoch [3/3], Step [4710/6471], Loss: 1.9688, Perplexity: 7.1622sel length 10


Epoch [3/3], Step [4910/6471], Loss: 2.2848, Perplexity: 9.8238sel length 11
Epoch [3/3], Step [4911/6471], Loss: 1.8144, Perplexity: 6.1374sel length 11
Epoch [3/3], Step [4912/6471], Loss: 1.9206, Perplexity: 6.8251sel length 13
Epoch [3/3], Step [4913/6471], Loss: 2.0385, Perplexity: 7.6794sel length 10
Epoch [3/3], Step [4914/6471], Loss: 1.7829, Perplexity: 5.9472sel length 13
Epoch [3/3], Step [4915/6471], Loss: 2.0587, Perplexity: 7.8360sel length 16
Epoch [3/3], Step [4916/6471], Loss: 2.3547, Perplexity: 10.5351sel length 10
Epoch [3/3], Step [4917/6471], Loss: 1.8287, Perplexity: 6.2255sel length 11
Epoch [3/3], Step [4918/6471], Loss: 1.7872, Perplexity: 5.9728sel length 9
Epoch [3/3], Step [4919/6471], Loss: 1.9862, Perplexity: 7.2880sel length 9
Epoch [3/3], Step [4920/6471], Loss: 2.0106, Perplexity: 7.4679sel length 9
Epoch [3/3], Step [4921/6471], Loss: 1.9803, Perplexity: 7.2447sel length 10
Epoch [3/3], Step [4922/6471], Loss: 1.7417, Perplexity: 5.7069sel length 16
E

Epoch [3/3], Step [5122/6471], Loss: 1.9222, Perplexity: 6.8360sel length 11
Epoch [3/3], Step [5123/6471], Loss: 1.8925, Perplexity: 6.6356sel length 14
Epoch [3/3], Step [5124/6471], Loss: 2.0216, Perplexity: 7.5506sel length 12
Epoch [3/3], Step [5125/6471], Loss: 1.9007, Perplexity: 6.6904sel length 11
Epoch [3/3], Step [5126/6471], Loss: 1.8193, Perplexity: 6.1674sel length 10
Epoch [3/3], Step [5127/6471], Loss: 1.8977, Perplexity: 6.6702sel length 10
Epoch [3/3], Step [5128/6471], Loss: 1.8348, Perplexity: 6.2636sel length 14
Epoch [3/3], Step [5129/6471], Loss: 2.1340, Perplexity: 8.4483sel length 11
Epoch [3/3], Step [5130/6471], Loss: 1.8867, Perplexity: 6.5974sel length 11
Epoch [3/3], Step [5131/6471], Loss: 1.9407, Perplexity: 6.9633sel length 11
Epoch [3/3], Step [5132/6471], Loss: 1.8783, Perplexity: 6.5421sel length 8
Epoch [3/3], Step [5133/6471], Loss: 2.0940, Perplexity: 8.1171sel length 9
Epoch [3/3], Step [5134/6471], Loss: 1.8115, Perplexity: 6.1194sel length 9
Ep

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.